In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split

In [22]:
train = pd.read_csv("NEW_DATA/train.csv")
test = pd.read_csv("NEW_DATA/test.csv")

In [15]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28563 entries, 0 to 28562
Data columns (total 59 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                28563 non-null  int64  
 1   units                28563 non-null  int64  
 2   OP_Athens            28563 non-null  int64  
 3   OP_Barcelona         28563 non-null  int64  
 4   OP_Rotterdam         28563 non-null  int64  
 5   3pl_v_001            28563 non-null  int64  
 6   3pl_v_002            28563 non-null  int64  
 7   3pl_v_003            28563 non-null  int64  
 8   3pl_v_004            28563 non-null  int64  
 9   CP_CRF               28563 non-null  int64  
 10  CP_DTD               28563 non-null  int64  
 11  CP_DTP               28563 non-null  int64  
 12  LH_Bratislava        28563 non-null  int64  
 13  LH_Dusseldorf        28563 non-null  int64  
 14  LH_Hamburg           28563 non-null  int64  
 15  LH_Liege             28563 non-null 

In [3]:
delayed = train[train.late_order == True]
non_delayed = train[train.late_order == False]
shuffled = list(non_delayed.index)
random.shuffle(shuffled)
items = len(shuffled)//3

trucation_1 = shuffled[:items]
trucation_2 = shuffled[items:items*2]
trucation_3 = shuffled[items*2:]
df_part1 = non_delayed[non_delayed.index.isin(trucation_1)]
df_part2 = non_delayed[non_delayed.index.isin(trucation_2)]
df_part3 = non_delayed[non_delayed.index.isin(trucation_3)]
print(len(df_part1), len(df_part2), len(df_part3))
print(len(delayed))

29040 29040 29040
27156


In [4]:
df1 = pd.concat([df_part1, delayed])
df2 = pd.concat([df_part2, delayed])
df3 = pd.concat([df_part3, delayed])

df1.dropna(inplace=True)
df2.dropna(inplace=True)
df3.dropna(inplace=True)

In [5]:
train1, test1 = train_test_split(df1, test_size=0.2)

In [6]:
train1_x = train1.drop(columns=["late_order"])
train1_y = train1["late_order"]
test1_x = test1.drop(columns=["late_order"])
test1_y = test1["late_order"]

In [7]:
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

2022-11-13 04:15:44.472871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 04:15:44.620259: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-13 04:15:44.620284: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-13 04:15:44.655292: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-13 04:15:45.471787: W tensorflow/stream_executor/platform/de

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, input_shape = (train1_x.shape[1],), activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
model.fit(x=train1_x,y=train1_y, epochs = 30, validation_data=(test1_x, test1_y))

Epoch 1/30
1391/1391 [==============================] - 5s 3ms/step - loss: 0.7023 - accuracy: 0.5974 - val_loss: 0.6353 - val_accuracy: 0.6387
Epoch 2/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.6234 - accuracy: 0.6512 - val_loss: 0.6063 - val_accuracy: 0.6742
Epoch 3/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.6040 - accuracy: 0.6687 - val_loss: 0.5955 - val_accuracy: 0.6752
Epoch 4/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.5933 - accuracy: 0.6787 - val_loss: 0.5783 - val_accuracy: 0.6843
Epoch 5/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.5906 - accuracy: 0.6788 - val_loss: 0.6135 - val_accuracy: 0.6628
Epoch 6/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.5859 - accuracy: 0.6852 - val_loss: 0.5709 - val_accuracy: 0.6992
Epoch 7/30
1391/1391 [==============================] - 4s 3ms/step - loss: 0.5806 - accuracy: 0.6875 - val_loss: 0.5816 - val_accuracy:

In [46]:
result = model.predict(test.drop(columns=["order_id"]))

893/893 [==============================] - 1s 1ms/step


In [47]:
result = [r[0] for r in result]

submission = pd.DataFrame({
    "order_id":list(test["order_id"]),
    "late_order":list(result)
})

submission.fillna(0, inplace=True)

submission.to_csv("result.csv", index=False)

In [48]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28563 entries, 0 to 28562
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_id    28563 non-null  object 
 1   late_order  28563 non-null  float32
dtypes: float32(1), object(1)
memory usage: 334.8+ KB
